<a href="https://colab.research.google.com/github/fppfurtado/mvp-pos-ciencia-de-dados/blob/main/Projeto_MVP_Preditor_de_Pre%C3%A7os_de_T%C3%ADtulos_P%C3%BAblicos_Federais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Definição do Problema

## Objetivo
O presente projeto tem como objetivo a construção de um modelo de regressão linear para prever as taxas de juros ofertadas por títulos públicos federais - e, consequentemente, seus preços unitários - a partir dos valores da taxa de câmbio dolar/real (Ptax), do índice IBOVESPA e do CDS Brasil 5 anos.

## Títulos Públicos Federais
Títulos Públicos Federais são títulos de dívida emitidos pelo Governo Federal para financiar suas atvidades captando recursos de particulares em troca de pagamento de juros sobre um capital aplicado. Na prática, são **ativos de renda fixa** que possuem preço, taxa e vencimento definidos, e que podem ser adquiridos por investidores interessados.

Esses ativos possuem liquidez diária, podendo o investidor solicitar o resgate a qualquer tempo (durante o horário de funcionamento do mercado de títulos publicos), porém nesse caso o valor utilizado para o cálculo da rentabilidade será o **valor de mercado do título** no dia da solicitação de resgate, a chamada **Marcação a Mercado**, que pode fazer com que o investidor tenha ou lucro acima do esperado ou prejuízo no resgate antecipado.

O preço dos títulos públicos variam em função das respectivas taxas ofertadas. E as taxas variam diariamente, basicamente, em função da percepção de risco do mercado em relação à economia do país. Por isso, após alguma notícia negativa no cenário econômico, é comum observar fenômenos como "queda na bolsa", "alta no dolar" e "alta nos juros" pagos pelos títulos públicos federais.

## IBOVESPA
O Ibovespa é o principal indicador de desempenho das ações negociadas na B3 e reúne as empresas mais importantes do mercado de capitais brasileiro. Foi criado em 1968 e, ao longo desses 50 anos, consolidou-se como referência para investidores ao redor do mundo.

Reavaliado a cada quatro meses, o índice é resultado de uma carteira teórica de ativos. É composto pelas ações e units de companhias listadas na B3 que atendem aos critérios descritos na sua metodologia, correspondendo a cerca de 80% do número de negócios e do volume financeiro do nosso mercado de capitais.

Fonte: https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-amplos/ibovespa.htm

## PTax
Ptax é uma taxa de câmbio calculada durante o dia pelo Banco Central do Brasil. Consiste na média das taxas informadas pelas instituições financeiras durante 4 janelas do dia. É a taxa de referencia para o valor do dólar de D2 (em dois dias úteis). Normalmente, os contratos de derivativos de câmbio são liquidados com base na PTAX divulgada para o dia útil anterior.

Fonte: https://pt.wikipedia.org/wiki/Ptax

## CDS Brasil
CDS (credit default swap) é a troca de risco de crédito. O CDS é um contrato derivativo que funciona como um seguro para quem empresta dinheiro. Por levar em sua precificação o risco de inadimplência do tomador do empréstimo, o CDS é muito usado como termômetro para o risco de um país.

Fonte: https://inteligenciafinanceira.com.br/glossario/credit-default-swap-cds-risco-pais/

# 2. Coleta e Análise de Dados

## Instalação de Pacotes

In [ ]:
# Apenas na primeira execução
#!pip install investpy

## Importação de Pacotes e Módulos

In [ ]:
# Importando pacotes
import pandas as pd
#import investpy as inv
import json
import warnings
from urllib.request import urlopen
from google.colab import drive

## Configurações prévias

In [ ]:
# Suprimindo warnings
warnings.filterwarnings('ignore')
# Montando o Drive na pasta virtual do Colab
drive.mount('/content/drive')

Mounted at /content/drive


## Carregamento dos Dados

Serão carregados os datasets dos dados históricos de cinco anos das quatro variáveis econômicas que serão tratadas posteriormente pelo modelo de regressão linear: 

+ taxas e preços de título públicos federais
+ taxa de câmbio PTax
+ índice IBOVESPA
+ CDS Brasil 5 anos.

In [ ]:
# Carregando o histórico de preços de preços e taxas de títulos públicos
historico_titulos = pd.read_csv('drive/MyDrive/PROJETOS/PÓS CIENCIA DE DADOS/DADOS/historico_titulos.csv', decimal = ',')
# Carregando o histórico da taxa de câmbio PTax
historico_ptax = pd.read_csv('drive/MyDrive/PROJETOS/PÓS CIENCIA DE DADOS/DADOS/historico_ptax.csv', decimal = ',')
# Carregando o histórico do índice IBOVESPA
historico_ibov = pd.read_csv('drive/MyDrive/PROJETOS/PÓS CIENCIA DE DADOS/DADOS/historico_ibov.csv', decimal = ',')
# Carregando o histórico do CDS Brasil 5 Anos
historico_cds5y = pd.read_csv('drive/MyDrive/PROJETOS/PÓS CIENCIA DE DADOS/DADOS/historico_csd5y.csv', decimal = ',')

## Análise Exploratória

In [ ]:
historico_ibov.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1239 entries, 0 to 1238
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   direction_color    1239 non-null   object 
 1   rowDate            1239 non-null   object 
 2   rowDateRaw         1239 non-null   int64  
 3   rowDateTimestamp   1239 non-null   object 
 4   last_close         1239 non-null   float64
 5   last_open          1239 non-null   float64
 6   last_max           1239 non-null   float64
 7   last_min           1239 non-null   float64
 8   volume             1239 non-null   object 
 9   volumeRaw          1239 non-null   int64  
 10  change_precent     1239 non-null   object 
 11  last_closeRaw      1239 non-null   object 
 12  last_openRaw       1239 non-null   object 
 13  last_maxRaw        1239 non-null   object 
 14  last_minRaw        1239 non-null   object 
 15  change_precentRaw  1239 non-null   object 
dtypes: float64(4), int64(2),